### ----- Import Requirements -----

In [1]:
import pandas as pd
import numpy as np

### ----- Define Functions -----

In [32]:
# five allocation models

#last touchpoint
def find_last_touchpoint(t_row, touch_col_prepend_f, max_touches_f):
    for t in range(max_touches_f, 0, -1):
        t_val = t_row[touch_col_prepend_f + str(t)]
        try:
            if isinstance(t_val, str):
                return t_val
        except:
            None
    return None

# first touchpoint
def find_first_touchpoint(t_row, touch_col_prepend_f, max_touches_f):
    for t in range(1, max_touches_f + 1):
        t_val = t_row[touch_col_prepend_f + str(t)]
        try:
            if isinstance(t_val, str):
                return t_val
        except:
            None
    return None

#last non-direct touch point
def find_last_nondirect_touchpoint(t_row, touch_col_prepend_f, max_touches_f, direct_label_f):
    for t in range(max_touches_f, 0, -1):
        t_val = t_row[touch_col_prepend_f + str(t)]
        try:
            if isinstance(t_val, str) and t_val != direct_label_f:
                return t_val
        except:
            None
    return None

In [33]:
def assign_credit(t_row, cred_col_names_f, touch_col_names_f, cred_col_post_pend_f, model_type_f, first_weight_f=0.5, last_weight_f=0.5):
    # function assigns a credit to each relevant channel based on user specified model type, e.g. "last_touch_point", "first_touch_point", etc.
    t_dict = dict(zip(cred_col_names_f, [0]*len(cred_col_names_f)))

    if model_type_f == 'last_touch_point':
        # last
        t_dict.update({t_row['last_touch_point'] + cred_col_post_pend_f: 1})
        return t_dict
    elif model_type_f == 'first_touch_point':
        # first
        t_dict.update({t_row['first_touch_point'] + cred_col_post_pend_f: 1})
        return t_dict
    elif model_type_f == 'last_nondirect_touch_point':
        # last_non_direct
        try:
            t_dict.update(
                {t_row['last_nondirect_touch_point'] + cred_col_post_pend_f: 1})
            return t_dict
        except TypeError:
            # case where there is no other channel
            t_dict.update({'direct' + cred_col_post_pend_f: 1})
            return t_dict
    elif (model_type_f == 'linear') or (model_type_f == 'position'):
        # linear and position based
        t_channels = [x for x in t_row[touch_col_names_f]
                      if isinstance(x, str)]
        if model_type_f == 'linear':
            # linear weights
            t_weights = [1 / len(t_channels)] * len(t_channels)
        elif model_type_f == 'position':
            # position based weights (first and last specified, middle divided evenly)
            if len(t_channels) > 2:
                t_weights = [first_weight_f] + [(1 - (first_weight_f + last_weight_f)) / (
                    len(t_channels) - 2)] * (len(t_channels) - 2) + [last_weight_f]
            elif len(t_channels) == 1:
                t_weights = [1]
            else:
                t_weights = [first_weight_f] + [last_weight_f]

        t_weights = [x / sum(t_weights)
                     for x in t_weights]     # ensure weights sum to 1
        for i in range(0, len(t_weights)):
            t_key = t_channels[i] + '_credit'
            t_value = t_dict[t_key] + t_weights[i]
            t_dict.update({t_key: t_value})
        return t_dict
    else:
        return t_dict

In [34]:
def get_attribution_by_channel(df_f, credit_col_postpend_f):
    allocated_conversions = df_f[cred_col_names].sum()  # sum of each credit
    n_allocated_conversions = df_f[cred_col_names].sum().sum()  # sum of all credits
    n_total_conversions = df_f.convert_TF.sum()   # number of Trues
    if n_allocated_conversions != n_total_conversions:
        print('WARNING: allocation error. Sum of allocated conversions = %d. Sum of total conversions = %d' % (
            int(n_allocated_conversions), int(n_total_conversions)))

    channel_allocation_f = pd.Series(dict(zip([x.split(credit_col_postpend_f)[
                                     0] for x in allocated_conversions.keys()], list(allocated_conversions.array))))
    return channel_allocation_f

In [35]:
def calc_avg_CAC(channel_allocation_f, channel_spend_f):
    t_df = pd.DataFrame(channel_allocation_f)
    t_df.columns = ['channel_allocation']
    for t_ind, _ in t_df.iterrows():
        t_df.loc[t_ind, 'channel_spend'] = channel_spend_f[t_ind]
    t_df['CAC'] = t_df['channel_spend'] / t_df['channel_allocation']
    t_df['CAC'].replace(np.inf, 0, inplace=True)
    return t_df

In [ ]:
def calc_marginal_CAC(n_conversions_low_tier, spend_low_tier, n_conversions_high_tier, spend_high_tier):
    # fill in this code to create the three variables in output dictionary
    return {'marginal_conversions': marginal_conversions, 'marginal_spend': marginal_spend,
            'marginal_CAC': marginal_CAC}

`get_attribution_by_channel`

In [75]:
#allocated_conversions
df[cred_col_names].sum()

paid_search_credit       0
email_credit             0
organic_search_credit    0
referral_credit          0
social_credit            0
display_credit           0
direct_credit            0
dtype: int64

In [69]:
# n_allocated_conversions
df[cred_col_names].sum().sum()

0

In [70]:
# n_total_conversions
df.convert_TF.sum()

13779

In [71]:
df.convert_TF.value_counts()

False    54345
True     13779
Name: convert_TF, dtype: int64

In [86]:
# channel_allocation_f
#pd.Series(dict(zip([x.split(credit_col_postpend_f)[0] for x in allocated_conversions.keys()], 
                    #list(allocated_conversions.array))))

In [67]:
list(df[cred_col_names].sum().array)

[0, 0, 0, 0, 0, 0, 0]

In [76]:
df[cred_col_names].sum().keys() # cred_col_names

Index(['paid_search_credit', 'email_credit', 'organic_search_credit',
       'referral_credit', 'social_credit', 'display_credit', 'direct_credit'],
      dtype='object')

In [81]:
[x.split('_credit')[0] for x in cred_col_names]

['paid_search',
 'email',
 'organic_search',
 'referral',
 'social',
 'display',
 'direct']

In [83]:
dict(zip([x.split('_credit')[0] for x in cred_col_names], list(df[cred_col_names].sum().array)))

{'paid_search': 0,
 'email': 0,
 'organic_search': 0,
 'referral': 0,
 'social': 0,
 'display': 0,
 'direct': 0}

### ----- Set parameters -----

In [2]:
touch_col_prepend = 'touch'
direct_label = 'direct'
first_weight = 0.4
last_weight = 0.4
cred_col_post_pend = '_credit'
select_model_types = ['last_touch_point', 'first_touch_point',
                      'last_nondirect_touch_point', 'linear', 'position']
write_to_file = True

### ----- Import data -----

In [3]:
df = pd.read_pickle('attribution_allocation_student_data')
channel_spend = pd.read_pickle('channel_spend_student_data')

I had problem inspecting the pickle data but it worked after updating `pandas` to version 1.1.1

In [6]:
df.head(1)

,convert_TF,touch1,touch2,touch3,touch4,touch5,tier
0,True,referral,referral,referral,email,NaN,1


In [8]:
channel_spend

tier1    {'email': 1000.0, 'social': 1000.0000000000001...
tier2    {'email': 2000.0, 'social': 2000.0, 'display':...
tier3    {'email': 3000.0, 'social': 3000.0, 'display':...
total    {'email': 5000.0, 'social': 5000.0, 'display':...
dtype: object

In [19]:
channel_spend['tier1']

{'email': 1000.0,
 'social': 1000.0000000000001,
 'display': 1000.0,
 'paid_search': 999.9999999999999,
 'referral': 1000.0,
 'organic_search': 0.0,
 'direct': 0.0}

In [96]:
df = df[:20].copy()

In [97]:
df.shape

(20, 17)

### ----- Calculations -----

In [9]:
touch_col_names = [x for x in df.columns 
                   if x.find(touch_col_prepend) > -1]

max_touches = max([int(x.split(touch_col_prepend)[1])
                   for x in touch_col_names])

# total spending for all three tier experiments
channel_spend['total'] = dict()
for t_name, t in channel_spend.items():
    if t_name != 'total':   # tiers
        for c in t.keys():  # channels
            try:
                channel_spend['total'][c] = channel_spend['total'][c] + t[c]
            except KeyError:
                channel_spend['total'].update({c: 0})

In [10]:
touch_col_names

['touch1', 'touch2', 'touch3', 'touch4', 'touch5']

In [11]:
max_touches

5

In [12]:
channel_spend['total']

{'email': 5000.0,
 'social': 5000.0,
 'display': 5000.0,
 'paid_search': 5000.0,
 'referral': 5000.0,
 'organic_search': 0.0,
 'direct': 0.0}

### ----- Format dataframe -----

In [21]:
# --- create credit columns
base_set = set()
[base_set.update(set(df[x].dropna().unique())) for x in touch_col_names]
cred_col_names = [x + '_credit' for x in base_set]
df = pd.concat([df, pd.DataFrame(data=0, columns=cred_col_names,
                                 index=df.index)], axis=1, ignore_index=False)

In [23]:
base_set

{'direct',
 'display',
 'email',
 'organic_search',
 'paid_search',
 'referral',
 'social'}

In [24]:
cred_col_names

['paid_search_credit',
 'email_credit',
 'organic_search_credit',
 'referral_credit',
 'social_credit',
 'display_credit',
 'direct_credit']

In [26]:
df.head()

,convert_TF,touch1,touch2,touch3,touch4,touch5,tier,paid_search_credit,email_credit,organic_search_credit,referral_credit,social_credit,display_credit,direct_credit
0,True,referral,referral,referral,email,NaN,1,0,0,0,0,0,0,0
1,True,referral,display,display,email,NaN,1,0,0,0,0,0,0,0
2,True,email,display,referral,NaN,NaN,1,0,0,0,0,0,0,0
3,True,referral,referral,email,NaN,NaN,1,0,0,0,0,0,0,0
4,True,social,referral,NaN,NaN,NaN,1,0,0,0,0,0,0,0


In [27]:
df.shape

(68124, 14)

In [36]:
# --- identify key touch points
df['last_touch_point'] = df.apply(
    find_last_touchpoint, args=(touch_col_prepend, max_touches), axis=1)
df['first_touch_point'] = df.apply(
    find_first_touchpoint, args=(touch_col_prepend, max_touches), axis=1)
df['last_nondirect_touch_point'] = df.apply(find_last_nondirect_touchpoint, args=(
    touch_col_prepend, max_touches, direct_label,), axis=1)

In [38]:
df.head(1)

,convert_TF,touch1,touch2,touch3,touch4,touch5,tier,paid_search_credit,email_credit,organic_search_credit,referral_credit,social_credit,display_credit,direct_credit,last_touch_point,first_touch_point,last_nondirect_touch_point
0,True,referral,referral,referral,email,NaN,1,0,0,0,0,0,0,0,email,referral,email


In [ ]:
# ----- RUN MODELS -----
CAC_dfs = dict()
for model_type in select_model_types:
    print('Processing model %s' % model_type)

    # ----- Run attribution model -----
    print('Running attribution model')
    # only run calculation for conversion rows
    df_convert = df.loc[df.convert_TF]
    for t_ind, t_row in df_convert.iterrows():
        t_credit_dict = assign_credit(
            t_row, cred_col_names, touch_col_names, cred_col_post_pend, model_type, first_weight, last_weight)
        df.loc[t_ind, list(t_credit_dict.keys())] = list(t_credit_dict.values())  # add credit to original dataframe
    del df_convert, t_ind, t_row

    # ----- Calculate CAC -----
    print('Calculating average and marginal CAC')
    # --- Average CAC ---
    channel_allocation = get_attribution_by_channel(
        df, credit_col_postpend_f='_credit')
    df_CAC = calc_avg_CAC(channel_allocation_f=channel_allocation,
                          channel_spend_f=channel_spend['total'])

    # --- Marginal CAC ---
    credit_cols = [x for x in df.columns if x.find('credit') > -1]
    df_CAC = pd.DataFrame(index=[x.split('_credit')[0] for x in credit_cols])
    base_col_names = ['marginal_conversions', 'marginal_spend', 'marginal_CAC']

    df_tier_sum = df[['tier']+credit_cols].groupby(['tier']).sum()
    df_tier_sum.columns = [x.split('_credit')[0] for x in df_tier_sum.columns]
    for t_tier in df_tier_sum.index:
        for t_channel in df_CAC.index:
            if t_tier > 1:
                n_conversions_low_tier = df_tier_sum.loc[t_tier - 1, t_channel]
                spend_low_tier = channel_spend['tier' + str(t_tier - 1)][t_channel]
                n_conversions_high_tier = df_tier_sum.loc[t_tier, t_channel]
                spend_high_tier = channel_spend['tier' + str(t_tier)][t_channel]
            else:
                n_conversions_low_tier = 0
                spend_low_tier = 0
                n_conversions_high_tier = df_tier_sum.loc[t_tier, t_channel]
                spend_high_tier = channel_spend['tier' + str(t_tier)][t_channel]

            t_df_CAC_colnames = [x + '_t' + str(t_tier) for x in base_col_names]
            t_marginal_dict = calc_marginal_CAC(
                n_conversions_low_tier, spend_low_tier, n_conversions_high_tier, spend_high_tier)
            df_CAC.loc[t_channel, t_df_CAC_colnames] = [
                t_marginal_dict[x] for x in base_col_names]

    CAC_dfs.update({model_type: df_CAC})

testing (a small sample size) with "last_touch_point" model

In [88]:
select_model_types

['last_touch_point',
 'first_touch_point',
 'last_nondirect_touch_point',
 'linear',
 'position']

In [122]:
# ----- Run attribution model -----
print('Running attribution model')
# only run calculation for conversion rows
df_convert = df.loc[df.convert_TF]
for t_ind, t_row in df_convert.iterrows():
    t_credit_dict = assign_credit(
        t_row, cred_col_names, touch_col_names, cred_col_post_pend, 'last_touch_point', first_weight, last_weight)
    df.loc[t_ind, list(t_credit_dict.keys())] = list(t_credit_dict.values())  # add credit to original dataframe
del df_convert, t_ind, t_row

Running attribution model


In [123]:
df.head(1)

,convert_TF,touch1,touch2,touch3,touch4,touch5,tier,paid_search_credit,email_credit,organic_search_credit,referral_credit,social_credit,display_credit,direct_credit,last_touch_point,first_touch_point,last_nondirect_touch_point
0,True,referral,referral,referral,email,NaN,1,0,1,0,0,0,0,0,email,referral,email


In [124]:
# ----- Calculate CAC -----
print('Calculating average and marginal CAC')
# --- Average CAC ---
channel_allocation = get_attribution_by_channel(df, credit_col_postpend_f='_credit')
df_CAC = calc_avg_CAC(channel_allocation_f=channel_allocation, channel_spend_f=channel_spend['total'])

Calculating average and marginal CAC


In [125]:
channel_allocation

paid_search       1004
email             1491
organic_search      11
referral          6014
social            2960
display           2292
direct               7
dtype: int64

In [126]:
df_CAC

,channel_allocation,channel_spend,CAC
paid_search,1004,5000.0,4.980080
email,1491,5000.0,3.353454
organic_search,11,0.0,0.000000
referral,6014,5000.0,0.831393
social,2960,5000.0,1.689189
display,2292,5000.0,2.181501
direct,7,0.0,0.000000


In [ ]:
# --- Marginal CAC ---
credit_cols = [x for x in df.columns if x.find('credit') > -1]
df_CAC = pd.DataFrame(index=[x.split('_credit')[0] for x in credit_cols])
base_col_names = ['marginal_conversions', 'marginal_spend', 'marginal_CAC']

df_tier_sum = df[['tier']+credit_cols].groupby(['tier']).sum()
df_tier_sum.columns = [x.split('_credit')[0] for x in df_tier_sum.columns]
for t_tier in df_tier_sum.index:
    for t_channel in df_CAC.index:
        if t_tier > 1:
            n_conversions_low_tier = df_tier_sum.loc[t_tier - 1, t_channel]
            spend_low_tier = channel_spend['tier' + str(t_tier - 1)][t_channel]
            n_conversions_high_tier = df_tier_sum.loc[t_tier, t_channel]
            spend_high_tier = channel_spend['tier' + str(t_tier)][t_channel]
        else:
            n_conversions_low_tier = 0
            spend_low_tier = 0
            n_conversions_high_tier = df_tier_sum.loc[t_tier, t_channel]
            spend_high_tier = channel_spend['tier' + str(t_tier)][t_channel]

        t_df_CAC_colnames = [x + '_t' + str(t_tier) for x in base_col_names]
        t_marginal_dict = calc_marginal_CAC(n_conversions_low_tier, spend_low_tier, n_conversions_high_tier, spend_high_tier)
        df_CAC.loc[t_channel, t_df_CAC_colnames] = [t_marginal_dict[x] for x in base_col_names]

In [127]:
credit_cols = [x for x in df.columns if x.find('credit') > -1]
credit_cols

['paid_search_credit',
 'email_credit',
 'organic_search_credit',
 'referral_credit',
 'social_credit',
 'display_credit',
 'direct_credit']

In [128]:
pd.DataFrame(index=[x.split('_credit')[0] for x in credit_cols])

""
paid_search
email
organic_search
referral
social
display
direct


In [129]:
base_col_names = ['marginal_conversions', 'marginal_spend', 'marginal_CAC']
base_col_names

['marginal_conversions', 'marginal_spend', 'marginal_CAC']

In [130]:
df_tier_sum = df[['tier']+credit_cols].groupby(['tier']).sum()
df_tier_sum

,paid_search_credit,email_credit,organic_search_credit,referral_credit,social_credit,display_credit,direct_credit
tier,,,,,,,
1,176,281,2,1211,625,528,1
2,347,555,4,2107,1060,818,1
3,481,655,5,2696,1275,946,5


In [131]:
df_tier_sum.index

Int64Index([1, 2, 3], dtype='int64', name='tier')

In [132]:
df_CAC.index

Index(['paid_search', 'email', 'organic_search', 'referral', 'social',
       'display', 'direct'],
      dtype='object')

In [ ]:
for t_tier in df_tier_sum.index:
    for t_channel in df_CAC.index:
        if t_tier > 1:
            n_conversions_low_tier = df_tier_sum.loc[t_tier - 1, t_channel]
            spend_low_tier = channel_spend['tier' + str(t_tier - 1)][t_channel]
            n_conversions_high_tier = df_tier_sum.loc[t_tier, t_channel]
            spend_high_tier = channel_spend['tier' + str(t_tier)][t_channel]
        else:  # t_tier=1
            n_conversions_low_tier = 0
            spend_low_tier = 0
            n_conversions_high_tier = df_tier_sum.loc[t_tier, t_channel]
            spend_high_tier = channel_spend['tier' + str(t_tier)][t_channel]

        t_df_CAC_colnames = [x + '_t' + str(t_tier) for x in base_col_names]
        t_marginal_dict = calc_marginal_CAC(n_conversions_low_tier, spend_low_tier, n_conversions_high_tier, spend_high_tier)
        df_CAC.loc[t_channel, t_df_CAC_colnames] = [t_marginal_dict[x] for x in base_col_names]

In [137]:
# n_conversions_low_tier = df_tier_sum.loc[t_tier - 1, t_channel]
#df_tier_sum.loc[1, 'paid_search_credit']  # 176
df_tier_sum

,paid_search_credit,email_credit,organic_search_credit,referral_credit,social_credit,display_credit,direct_credit
tier,,,,,,,
1,176,281,2,1211,625,528,1
2,347,555,4,2107,1060,818,1
3,481,655,5,2696,1275,946,5


In [144]:
#channel_spend['tier' + str(t_tier - 1)][t_channel]
#channel_spend['tier' + str(1)]['paid_search']  # 999
channel_spend

tier1    {'email': 1000.0, 'social': 1000.0000000000001...
tier2    {'email': 2000.0, 'social': 2000.0, 'display':...
tier3    {'email': 3000.0, 'social': 3000.0, 'display':...
total    {'email': 5000.0, 'social': 5000.0, 'display':...
dtype: object

In [138]:
[x + '_t' + str(1) for x in base_col_names]

['marginal_conversions_t1', 'marginal_spend_t1', 'marginal_CAC_t1']